# Библиотеки Python для DS (семинары)

Урок 8. На практике использование методов генерации признаков

В вашем распоряжении набор данных о качестве сна Sleep_health_and_lifestyle_dataset.csv.
Ваша задача — подготовить данные к моделированию, сгенерировать новые признаки, применить методы отбора признаков для улучшения качества предсказательной модели и подобрать оптимальные гиперпараметры для модели регрессии.

1. Предобработка данных:
— Очистите данные от пропусков и аномалий.
— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

2. Генерация новых признаков:
— Создайте комбинированные признаки, которые могут помочь улучшить модель.
— Используйте полиномиальные признаки для числовых переменных.

3. Отбор признаков:
— Примените несколько методов отбора признаков (например, RFE, SelectKBest).
— Сравните качество модели до и после отбора признаков.

4. Подбор гиперпараметров:
— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.
— Оцените, как изменение гиперпараметров влияет на качество предсказаний.

## Выполнение домашней работы

##### Используем предоставленый к заданию файл fetch_california_housing.xlsx

In [67]:
# Библиотеки для работы с датасетом и графиками
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
# Загружаем датасет
df = pd.read_excel('fetch_california_housing.xlsx')

##### 1. Предобработка данных:

— Очистите данные от пропусков и аномалий.

— Преобразуйте категориальные переменные с помощью One-Hot Encoding или Label Encoding.

In [3]:
# Выводим дата фрейм
df.head()
df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41,6.984127,1.023810,322,2.555556,37.88,-122.23,4.526
1,8.3014,21,6.238137,0.971880,2401,2.109842,37.86,-122.22,3.585
2,7.2574,52,8.288136,1.073446,496,2.802260,37.85,-122.24,3.521
3,5.6431,52,5.817352,1.073059,558,2.547945,37.85,-122.25,3.413
4,3.8462,52,6.281853,1.081081,565,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25,5.045455,1.133333,845,2.560606,39.48,-121.09,0.781
20636,2.5568,18,6.114035,1.315789,356,3.122807,39.49,-121.21,0.771
20637,1.7000,17,5.205543,1.120092,1007,2.325635,39.43,-121.22,0.923
20638,1.8672,18,5.329513,1.171920,741,2.123209,39.43,-121.32,0.847


Этот фрейм данных содержит следующие столбцы:

1. MedInc (Средний доход населения в квартале)
2. HouseAge (Средний возраст дома в квартале)
3. AveRooms (Средние количество команат)
4. AveBedrms (Средние количество спален)
5. Population (Население квартала)
6. AveOccup (Средняя заполняемость дома)
7. Latitude (Широта жилого квартала)
8. Longitude (Долгота жилого квартала)
9. target (Средняя цена дома)

In [4]:
# Выводим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      20640 non-null  float64
 1   HouseAge    20640 non-null  int64  
 2   AveRooms    20640 non-null  float64
 3   AveBedrms   20640 non-null  float64
 4   Population  20640 non-null  int64  
 5   AveOccup    20640 non-null  float64
 6   Latitude    20640 non-null  float64
 7   Longitude   20640 non-null  float64
 8   target      20640 non-null  float64
dtypes: float64(7), int64(2)
memory usage: 1.4 MB


Из общей информации следует:
1. Всего 9 колонок
2. Нулевые значения отсуствуют
3. Нет пропущеных записей в колонках
4. Все колонки находятся в дробных float64 значениях, кроме "HouseAge" и "Population" они в int64

In [5]:
# Статистический анализ числовых переменных
df.describe().T

,count,mean,std,min,25%,50%,75%,max
MedInc,20640.0,3.870671,1.899822,0.499900,2.563400,3.534800,4.743250,15.000100
HouseAge,20640.0,28.639486,12.585558,1.000000,18.000000,29.000000,37.000000,52.000000
AveRooms,20640.0,5.429000,2.474173,0.846154,4.440716,5.229129,6.052381,141.909091
AveBedrms,20640.0,1.096675,0.473911,0.333333,1.006079,1.048780,1.099526,34.066667
Population,20640.0,1425.476744,1132.462122,3.000000,787.000000,1166.000000,1725.000000,35682.000000
AveOccup,20640.0,3.070655,10.386050,0.692308,2.429741,2.818116,3.282261,1243.333333
Latitude,20640.0,35.631861,2.135952,32.540000,33.930000,34.260000,37.710000,41.950000
Longitude,20640.0,-119.569704,2.003532,-124.350000,-121.800000,-118.490000,-118.010000,-114.310000
target,20640.0,2.068558,1.153956,0.149990,1.196000,1.797000,2.647250,5.000010


На основе представленных статистических данных неприрывных признаков можно сделать следующие выводы:

1. MedInc (Средний доход населения в квартале): 
- Средний доход населения на уровне 3.53
- Миниманая 0.49
- Максимальная 15.00
- Медиана равна 3.87
- Отклонение состовляет 1.89
2. HouseAge (Средний возраст дома в квартале):
- Средние средний возраст дома на уровне 29.00
- Миниманая 1.00
- Максимальная 52.00
- Медиана равна 28.63
- Отклонение состовляет 12.58
3. AveRooms (Средние количество комнат):
- Средние количество комнат на уровне 5.22
- Миниманая 0.84
- Максимальная 141.90
- Медиана равна 5.42
- Отклонение состовляет 2.47
4. AveBedrms (Средние количество спален):
- Средние количество спален на уровне 1.04
- Миниманая 0.33
- Максимальная 34.06
- Медиана равна 1.09
- Отклонение состовляет 0.47
5. Population (Население квартала):
- Средние население квартала на уровне 1166.00
- Миниманая 3.00
- Максимальная 35682.00
- Медиана равна 1425.47
- Отклонение состовляет 1132.46
6. AveOccup (Средняя заполняемость дома):
- Средние средняя заполняемость дома на уровне 2.81
- Миниманая 0.69
- Максимальная 1243.33
- Медиана равна 3.07
- Отклонение состовляет 10.38
7. Latitude (Широта жилого квартала):
- Средняя широта жилого квартала на уровне 34.26
- Миниманая 32.54
- Максимальная 41.95
- Медиана равна 35.63
- Отклонение состовляет 2.13
8. Longitude (Долгота жилого квартала):
- Средняя долгота жилого квартала на уровне -118.49
- Миниманая -124.35
- Максимальная -114.31
- Медиана равна -119.56
- Отклонение состовляет 2.00
9. target (Средняя цена дома)
- Средние цена дома на уровне 1.79
- Миниманая 0.14
- Максимальная 5.00
- Медиана равна 2.06
- Отклонение состовляет 1.15

In [6]:
# Проверяем на дубликаты
print("Количество дубликатов:")
print(df.duplicated().sum())

Количество дубликатов:
0


Дубликаты отсутствуют.

In [7]:
# Смотрим ещё раз тип данных
df.dtypes

MedInc        float64
HouseAge        int64
AveRooms      float64
AveBedrms     float64
Population      int64
AveOccup      float64
Latitude      float64
Longitude     float64
target        float64
dtype: object

In [8]:
# Проверка на нулевые значения
df.isnull().sum()

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
target        0
dtype: int64

In [9]:
# Проверка на NaN значения
df.isnull().any().any()

False

In [ ]:
# Установка стиля графиков
sns.set(style="whitegrid")

# Визуализация распределения нескольких выбранных числовых переменных
selected_columns = ['MedInc', 'HouseAge', 'AveRooms', 'AveOccup', 'target']

plt.figure(figsize=(15, 10))

for i, column in enumerate(selected_columns):
    plt.subplot(2, 3, i + 1)
    sns.histplot(df[column], kde=True, bins=30)
    plt.title(f'Распределение {column}')
    plt.xlabel(column)
    plt.ylabel('Частота')

plt.tight_layout()
plt.show()

In [ ]:
# Визуализация корреляции между выбранными переменными
correlation_matrix = df[selected_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Корреляционная матрица')
plt.show()

In [ ]:
correlation_matrix = df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, mask=mask, cmap='coolwarm', vmax=1, vmin=-1, center=0, 
            square=True, linewidths=.5, annot_kws={"size": 10}, 
            cbar_kws={"shrink": 0.8, "orientation": "vertical", "ticks": [-1, -0.5, 0, 0.5, 1]})
plt.title('Матрица корреляций между числовыми переменными')
plt.show()


##### 2. Генерация новых признаков:

— Создайте комбинированные признаки, которые могут помочь улучшить модель.

— Используйте полиномиальные признаки для числовых переменных.

In [13]:
df_home = df
# Извлечём diagnosis из признаков
X = df_home.drop('target', axis=1)
# Определим целевую переменную
target = df_home['target']

In [14]:
# Стандартизация данных
scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)

# Создание нового DataFrame со стандартизированными данными
df_scaled = pd.DataFrame(X_scaler, columns=X.columns)

In [15]:
# Определяем переменные для обучения
df_irus_scaled = df_scaled
y = target

# Разделяем данные на обучающий и тестовые наборы
X_train, X_test, y_train, y_test = train_test_split(df_irus_scaled, y, test_size=0.2, random_state=42)

# Выводим размеры
print("Размер обучающего набора:", X_train.shape)
print("Размер тестового набора:", X_test.shape)

Размер обучающего набора: (16512, 8)
Размер тестового набора: (4128, 8)


##### 3. Отбор признаков:

— Примените несколько методов отбора признаков (например, RFE, SelectKBest).

— Сравните качество модели до и после отбора признаков.

##### 4. Подбор гиперпараметров:

— Используйте GridSearchCV или RandomizedSearchCV для настройки параметров вашей модели регрессии.

— Оцените, как изменение гиперпараметров влияет на качество предсказаний.